# Quantum Phase Estimation algorithm (Q# code)

This notebook implements Quantum Phase Estimation algorithm (which uses Quantum Fourier Transform) in Q#.

See [here](https://tsmatz.wordpress.com/2019/04/26/quantum-computing-qsharp-quantum-fourier-transform-and-phase-estimation/) for mathematical background.

*back to [index](https://github.com/tsmatz/quantum-algorithms-qsharp)*

In [1]:
import qsharp

Preparing Q# environment...


In [2]:
# declare function
PhaseEstimationSample: any = None

In [3]:
%%qsharp
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;

operation QFTImpl (qs : Qubit[]) : Unit is Adj + Ctl
{
    body (...)
    {
        let nQubits = Length(qs);

        for i in 0 .. nQubits - 1
        {
            H(qs[i]);
            for j in i + 1 .. nQubits - 1
            {
                Controlled R1Frac([qs[j]], (1, j - i, qs[i]));
            }
        }

        Microsoft.Quantum.Canon.SwapReverseRegister(qs);
    }
}

operation QuantumPhaseEstimationImpl (oracle : (Qubit[] => Unit is Adj + Ctl), targetState : Qubit[], controlRegister : Qubit[]) : Unit is Adj + Ctl
{
    body (...)
    {
        let nQubits = Length(controlRegister);
        Microsoft.Quantum.Canon.ApplyToEachCA(H, controlRegister);

        for idxControlQubit in 0 .. nQubits - 1
        {
            let control = (controlRegister)[nQubits - 1 - idxControlQubit];
            let power = 2 ^ idxControlQubit;
            Controlled PowerOracle([control], (oracle, targetState, power));

            //// You can also write as follows,
            //// Or use Microsoft.Quantum.Canon.DiscreteOracle instead
            //for (idxPower in 0 .. power - 1)
            //{
            //  Controlled oracle([control], targetState);
            //}
        }

        Adjoint QFTImpl(controlRegister);
    }        
}

/// This is bloackbox oracle !
operation ExpOracle (eigenphase : Double, register : Qubit[]) : Unit is Adj + Ctl {        
    body (...) {
        Rz(2.0 * eigenphase, register[0]);
    }
}

operation PowerOracle (oracle : (Qubit[] => Unit is Adj + Ctl), targetState : Qubit[], power : Int) : Unit is Adj + Ctl {
    body (...) {
        for idxPower in 0 .. power - 1
        {
            oracle(targetState);
        }
    }
}

operation PhaseEstimationSample (eigenphase : Double) : Double {
    let oracle = ExpOracle(eigenphase, _); // Generate (Qubit[] => Unit) with eigenphase 
    let n = 10;

    mutable estPhase = 0.0;

    use (eigenstate, phaseRegister) = (Qubit[1], Qubit[n]) {
        X(eigenstate[0]);
        QuantumPhaseEstimationImpl(oracle, eigenstate, phaseRegister);
        let estReg = Microsoft.Quantum.Arithmetic.MeasureInteger(
            Microsoft.Quantum.Arithmetic.BigEndianAsLittleEndian(Microsoft.Quantum.Arithmetic.BigEndian(phaseRegister)));
        set estPhase = 2.0 * PI() * IntAsDouble(estReg) / IntAsDouble(2 ^ n);
        Reset(eigenstate[0]);
    }

    return estPhase;
}

In [4]:
import random

random.seed(1000)

eigenphase = random.uniform(0.0, 1.0) * 3.0 * 2.0
est = PhaseEstimationSample.simulate(eigenphase=eigenphase)
print("expected {}, estimated {}".format(eigenphase, est))

eigenphase = random.uniform(0.0, 1.0) * 3.0 * 2.0
est = PhaseEstimationSample.simulate(eigenphase=eigenphase)
print("expected {}, estimated {}".format(eigenphase, est))

eigenphase = random.uniform(0.0, 1.0) * 3.0 * 2.0
est = PhaseEstimationSample.simulate(eigenphase=eigenphase)
print("expected {}, estimated {}".format(eigenphase, est))

expected 4.664139856203383, estimated 4.663301595172349
expected 4.018953357355498, estimated 4.01902966426038
expected 0.5948376235489021, estimated 0.5951845456996288
